### ITEM-BASED COLLABORATIVE FILTERING

#### ПАКЕТSURPRISE

* используйте данные MovieLens 1M
https://grouplens.org/datasets/movielens/latest/

* можно использовать любые модели из пакета
* получите RMSE на тестовом сете 0.87 и ниже

In [1]:
!pip install scikit-surprise

In [2]:
from surprise import KNNWithMeans, KNNBasic , SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split



import pandas as pd

In [5]:
movies = pd.read_csv('movies.csv')
ratings_ = pd.read_csv('ratings.csv')

In [6]:
ratings = ratings_.sample(100000)

In [7]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [8]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [9]:
dataset.head()

,uid,iid,rating
0,167.0,Toy Story (1995),3.5
1,216.0,Toy Story (1995),3.0
2,414.0,Toy Story (1995),4.0
3,18.0,Toy Story (1995),3.5
4,514.0,Toy Story (1995),4.0


In [10]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)
# data_1 = Dataset.load_from_df(movies_with_ratings, reader)

In [11]:
trainset, testset = train_test_split(data, test_size=.15, random_state=42)

In [12]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [13]:
test_pred = algo.test(testset)

Результат, полученный с помощью KNNWithMeans хуже требуемого 

In [14]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8828


0.8827672719311593

In [15]:
# test_pred

In [16]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [17]:
test_pred = algo.test(testset)

In [18]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8724


0.8723635393720608

Применим SVD

In [19]:
%%time
algo = SVD(n_factors=20, n_epochs=30)
algo.fit(trainset)

Wall time: 5.16 s


In [20]:
test_pred = algo.test(testset)

In [21]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8598


0.8597763503477467

Результат, полученный с  помощью SVD, удовлетворяет условиям задания